In [4]:
#textFile = sc.textFile("file:///home/hadoop/Jiaoyan/ds504-project/data_format/user_log.csv")
#sample = textFile.sample(False, 0.001, 12345)
#sample.saveAsTextFile("file:///home/hadoop/Jiaoyan/ds504-project/data_format/writeback.txt")

In [42]:
from pyspark.sql import Row
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [43]:
filename = "file:///home/hadoop/Jiaoyan/ds504-project/part-00001"
textFile = sc.textFile(filename).map(lambda t: t.split(","))

# delete province
part = textFile.map(lambda t: (t[0],t[1],t[2],t[3],t[4],t[5],t[6],t[7],t[8],t[9]))
part.collect()

[('58467', '557857', '1611', '4845', '3345', '11', '07', '3', '2', '1'),
 ('35504', '299243', '266', '2699', '1487', '10', '22', '0', '5', '2'),
 ('11518', '382961', '720', '1071', '2100', '11', '11', '0', '0', '1'),
 ('41212', '631598', '1611', '1786', '7664', '11', '09', '3', '6', '2'),
 ('53692', '35800', '1208', '4990', '994', '10', '07', '0', '5', '2'),
 ('304064', '405357', '35', '2639', '7063', '09', '26', '0', '0', '0'),
 ('422987', '378861', '766', '1872', '3474', '11', '03', '3', '5', '1'),
 ('422987', '468396', '664', '2669', '3776', '11', '11', '0', '5', '2'),
 ('99382', '750876', '598', '4218', '3700', '11', '11', '0', '2', '2'),
 ('241180', '1083946', '420', '1', '1662', '07', '23', '0', '4', '0'),
 ('351262', '236114', '218', '2669', '3776', '05', '29', '0', '6', '2'),
 ('371128', '540438', '1208', '2576', '5580', '11', '10', '0', '3', '2'),
 ('210444', '1091896', '662', '2963', '6803', '06', '18', '0', '4', '1'),
 ('368696', '336741', '1028', '1', '1662', '11', '02', '0

In [44]:
# convert rdd to spark dataframe
data = part.map(lambda p : Row(user_id = p[0], item_id = p[1], cat_id = p[2], merchant_id = p[3], brand_id = p[4], \
                               month = p[5], day = p[6], action = p[7], age_range = int(p[8]), gender = p[9]))

# Infer the schema, and register the DataFrame as a table.
schemaData = spark.createDataFrame(data)
schemaData.createOrReplaceTempView("data")

schemaData.show()

+------+---------+--------+------+---+------+-------+-----------+-----+-------+
|action|age_range|brand_id|cat_id|day|gender|item_id|merchant_id|month|user_id|
+------+---------+--------+------+---+------+-------+-----------+-----+-------+
|     3|        2|    3345|  1611| 07|     1| 557857|       4845|   11|  58467|
|     0|        5|    1487|   266| 22|     2| 299243|       2699|   10|  35504|
|     0|        0|    2100|   720| 11|     1| 382961|       1071|   11|  11518|
|     3|        6|    7664|  1611| 09|     2| 631598|       1786|   11|  41212|
|     0|        5|     994|  1208| 07|     2|  35800|       4990|   10|  53692|
|     0|        0|    7063|    35| 26|     0| 405357|       2639|   09| 304064|
|     3|        5|    3474|   766| 03|     1| 378861|       1872|   11| 422987|
|     0|        5|    3776|   664| 11|     2| 468396|       2669|   11| 422987|
|     0|        2|    3700|   598| 11|     2| 750876|       4218|   11|  99382|
|     0|        4|    1662|   420| 23|  

In [45]:
# generate new features
# 1 Monthly click/add/favorite/purchase count for certain brand/ item/ category/ merchant
import pyspark.sql.functions as func
from pyspark.sql.functions import *
for_merchant = schemaData.groupBy(schemaData['merchant_id'],schemaData['month'], schemaData['action']).\
                            agg(count('action').alias("monthly_action_count_for_merchant"))
for_brand = schemaData.groupBy(schemaData['brand_id'],schemaData['month'], schemaData['action']).\
                            agg(count('action').alias("monthly_action_count_for_brand"))
for_category = schemaData.groupBy(schemaData['cat_id'],schemaData['month'], schemaData['action']).\
                            agg(count('action').alias("monthly_action_count_for_cat"))
for_item = schemaData.groupBy(schemaData['item_id'],schemaData['month'], schemaData['action']).\
                            agg(count('action').alias("monthly_action_count_for_item"))


joinDF = schemaData.join(for_merchant,["merchant_id","month","action"], "left_outer")
joinDF = joinDF.join(for_brand,["brand_id","month","action"], "left_outer")
joinDF = joinDF.join(for_category,["cat_id","month","action"], "left_outer")
joinDF = joinDF.join(for_item,["item_id","month","action"], "left_outer")
joinDF.show()

+-------+-----+------+------+--------+-----------+---------+---+------+-------+---------------------------------+------------------------------+----------------------------+-----------------------------+
|item_id|month|action|cat_id|brand_id|merchant_id|age_range|day|gender|user_id|monthly_action_count_for_merchant|monthly_action_count_for_brand|monthly_action_count_for_cat|monthly_action_count_for_item|
+-------+-----+------+------+--------+-----------+---------+---+------+-------+---------------------------------+------------------------------+----------------------------+-----------------------------+
|1084942|   11|     0|   748|    6741|       2179|        2| 10|     1|  88305|                                1|                             1|                           3|                            1|
| 294573|   09|     0|    66|    1439|       1056|        3| 28|     0|  80593|                                1|                             1|                           1|           

In [46]:
#2. User’s monthly action count of click/add/favorite/purchase for all merchant
for_user = schemaData.groupBy(schemaData['user_id'],schemaData['month'], schemaData['action']).\
                            agg(count('merchant_id').alias('user_monthly_action'))
joinDF = joinDF.join(for_user,['user_id','month','action'], 'left_outer')
joinDF.show()

+-------+-----+------+-------+------+--------+-----------+---------+---+------+---------------------------------+------------------------------+----------------------------+-----------------------------+-------------------+
|user_id|month|action|item_id|cat_id|brand_id|merchant_id|age_range|day|gender|monthly_action_count_for_merchant|monthly_action_count_for_brand|monthly_action_count_for_cat|monthly_action_count_for_item|user_monthly_action|
+-------+-----+------+-------+------+--------+-----------+---------+---+------+---------------------------------+------------------------------+----------------------------+-----------------------------+-------------------+
| 110468|   11|     0| 445277|  1611|    7890|       4009|        5| 11|     0|                                1|                             1|                           4|                            1|                  1|
| 263474|   08|     0| 172291|   351|    6585|       1892|        6| 19|     2|                         

In [47]:
#3.Days of a certains user clicked/added/favorited/purchased items in each month.
action_days = schemaData.groupBy(schemaData['user_id'],schemaData['action']).\
                            agg(count('day').alias('user_action_days'))
joinDF = joinDF.join(action_days,['user_id','action'], 'left_outer')
joinDF.show()

+-------+------+-----+-------+------+--------+-----------+---------+---+------+---------------------------------+------------------------------+----------------------------+-----------------------------+-------------------+----------------+
|user_id|action|month|item_id|cat_id|brand_id|merchant_id|age_range|day|gender|monthly_action_count_for_merchant|monthly_action_count_for_brand|monthly_action_count_for_cat|monthly_action_count_for_item|user_monthly_action|user_action_days|
+-------+------+-----+-------+------+--------+-----------+---------+---+------+---------------------------------+------------------------------+----------------------------+-----------------------------+-------------------+----------------+
| 112886|     0|   11|  19311|  1208|    4360|       4766|        7| 10|     1|                                2|                             2|                           9|                            1|                  1|               1|
| 142969|     0|   11| 350902|    13

In [48]:
#4.Total number of unique items/brand/category for a merchant
unique_items_for_merchant = schemaData.groupBy(schemaData['merchant_id']).\
                            agg(func.countDistinct('item_id').alias('unique_items_for_merchant'))
unique_cats_for_merchant = schemaData.groupBy(schemaData['merchant_id']).\
                            agg(func.countDistinct('cat_id').alias('unique_cats_for_merchant'))
unique_brands_for_merchant = schemaData.groupBy(schemaData['merchant_id']).\
                            agg(func.countDistinct('brand_id').alias('unique_brands_for_merchant'))

joinDF = joinDF.join(unique_items_for_merchant,['merchant_id'], 'left_outer')
joinDF = joinDF.join(unique_cats_for_merchant,['merchant_id'], 'left_outer')
joinDF = joinDF.join(unique_brands_for_merchant,['merchant_id'], 'left_outer')
joinDF.show()

+-----------+-------+------+-----+-------+------+--------+---------+---+------+---------------------------------+------------------------------+----------------------------+-----------------------------+-------------------+----------------+-------------------------+------------------------+--------------------------+
|merchant_id|user_id|action|month|item_id|cat_id|brand_id|age_range|day|gender|monthly_action_count_for_merchant|monthly_action_count_for_brand|monthly_action_count_for_cat|monthly_action_count_for_item|user_monthly_action|user_action_days|unique_items_for_merchant|unique_cats_for_merchant|unique_brands_for_merchant|
+-----------+-------+------+-----+-------+------+--------+---------+---+------+---------------------------------+------------------------------+----------------------------+-----------------------------+-------------------+----------------+-------------------------+------------------------+--------------------------+
|        296| 106083|     0|   06| 398285| 

In [49]:
#5.The number of unique merchant that the user clicked/purchased/added/favourites in each month
unique_merchants_in_each_month = schemaData.groupBy(schemaData['user_id'],schemaData['month'],schemaData['action'])\
                            .agg(func.countDistinct('merchant_id').alias('unique_merchants_in_each_month'))

joinDF = joinDF.join(unique_merchants_in_each_month,['user_id','month','action'], 'left_outer')
joinDF.show()

+-------+-----+------+-----------+-------+------+--------+---------+---+------+---------------------------------+------------------------------+----------------------------+-----------------------------+-------------------+----------------+-------------------------+------------------------+--------------------------+------------------------------+
|user_id|month|action|merchant_id|item_id|cat_id|brand_id|age_range|day|gender|monthly_action_count_for_merchant|monthly_action_count_for_brand|monthly_action_count_for_cat|monthly_action_count_for_item|user_monthly_action|user_action_days|unique_items_for_merchant|unique_cats_for_merchant|unique_brands_for_merchant|unique_merchants_in_each_month|
+-------+-----+------+-----------+-------+------+--------+---------+---+------+---------------------------------+------------------------------+----------------------------+-----------------------------+-------------------+----------------+-------------------------+------------------------+---------

In [50]:
#6. The number of unique user who clicked/purchased/added/favourites for a given items/brands/categories/merchant
unique_users_for_item = schemaData.groupBy(schemaData['item_id'],schemaData['action']).\
                        agg(func.countDistinct('user_id').alias('unique_users_for_item'))
    
unique_users_for_cat = schemaData.groupBy(schemaData['cat_id'],schemaData['action']).\
                        agg(func.countDistinct('user_id').alias('unique_users_for_cat'))
    
unique_users_for_brand= schemaData.groupBy(schemaData['brand_id'],schemaData['action']).\
                        agg(func.countDistinct('user_id').alias('unique_users_for_brand'))
    
unique_users_for_merchant = schemaData.groupBy(schemaData['merchant_id'],schemaData['action']).\
                        agg(func.countDistinct('user_id').alias('unique_users_for_merchant'))

joinDF = schemaData.join(unique_users_for_item,['item_id','action'], 'left_outer')
joinDF = joinDF.join(unique_users_for_cat,['cat_id','action'], 'left_outer')
joinDF = joinDF.join(unique_users_for_brand,['brand_id','action'], 'left_outer')
joinDF = joinDF.join(unique_users_for_merchant,['merchant_id','action'], 'left_outer')
joinDF.show()

+-----------+------+--------+------+-------+---------+---+------+-----+-------+---------------------+--------------------+----------------------+-------------------------+
|merchant_id|action|brand_id|cat_id|item_id|age_range|day|gender|month|user_id|unique_users_for_item|unique_users_for_cat|unique_users_for_brand|unique_users_for_merchant|
+-----------+------+--------+------+-------+---------+---+------+-----+-------+---------------------+--------------------+----------------------+-------------------------+
|       1818|     0|    3929|  1648| 321174|        2| 06|     1|   11| 160455|                    1|                   1|                     2|                        1|
|       4593|     0|    1742|   766|1073598|        4| 15|     0|   10| 102425|                    1|                   1|                     1|                        1|
|       2821|     0|    6236|   407| 241166|        3| 09|     2|   09| 104740|                    1|                  11|                  

In [22]:
#7.Counts of clicked/purchased/added/favorited of items for a certain user on the last Double 11 day
filtered = schemaData.filter((schemaData['month'] == 11) & (schemaData['day'] == 11))
action_on_double11 = filtered.groupBy(filtered['user_id'],filtered['action']).\
                        agg(func.countDistinct('action').alias('action_on_double11'))
join_double11 = schemaData.join(action_on_double11,['user_id','action'], 'left_outer')
join_double11 = join_double11.na.fill(0)
join_double11.show()

+-------+------+---------+--------+------+---+------+-------+-----------+-----+------------------+
|user_id|action|age_range|brand_id|cat_id|day|gender|item_id|merchant_id|month|action_on_double11|
+-------+------+---------+--------+------+---+------+-------+-----------+-----+------------------+
| 112886|     0|        7|    4360|  1208| 10|     1|  19311|       4766|   11|                 0|
| 142969|     0|        2|    3549|    13| 10|     1| 350902|       4881|   11|                 0|
| 382290|     0|        5|    5376|   662| 22|     2| 562245|       4173|   06|                 0|
| 195935|     0|        4|    3700|  1188| 06|     0| 570495|       4218|   11|                 0|
| 195935|     0|        6|    7649|  1349| 18|     1| 799796|       1503|   08|                 0|
|  29240|     2|        0|    7816|   632| 16|     2| 938197|        791|   10|                 0|
| 295282|     0|        5|    5801|  1112| 12|     1|1072341|       3094|   06|                 0|
| 406796| 

In [39]:
#8.1 The number of users who bought on at least two different days from items
filtered_purchase =  schemaData.filter(schemaData['action'] == 2)
purchase_item = filtered_purchase.groupBy(filtered_purchase['item_id'],filtered_purchase['user_id']).\
                            agg(func.countDistinct('day').alias('purchase_days'))
purchase_item_two_times = purchase_item.filter(purchase_item['purchase_days'] > 1)
user_purchase_item_two_times = purchase_item_two_times.groupBy(purchase_item_two_times['item_id']).\
                            agg(func.countDistinct('user_id').alias('user_purchase_item_two_times'))

join_item_purchase = schemaData.join(user_purchase_item_two_times,['item_id'], 'left_outer')
join_item_purchase = join_item_purchase.na.fill(0)

#8.2 The number of users who bought on at least two different days from brands
purchase_brand = filtered_purchase.groupBy(filtered_purchase['brand_id'],filtered_purchase['user_id']).\
                            agg(func.countDistinct('day').alias('purchase_days'))
purchase_brand_two_times = purchase_brand.filter(purchase_brand['purchase_days'] > 1)
user_purchase_brand_two_times = purchase_brand_two_times.groupBy(purchase_brand_two_times['brand_id']).\
                            agg(func.countDistinct('user_id').alias('user_purchase_brand_two_times'))

join_brand_purchase = join_item_purchase.join(user_purchase_brand_two_times,['brand_id'], 'left_outer')
join_brand_purchase = join_brand_purchase.na.fill(0)

#8.3 The number of users who bought on at least two different days from categories
purchase_cat = filtered_purchase.groupBy(filtered_purchase['cat_id'],filtered_purchase['user_id']).\
                            agg(func.countDistinct('day').alias('purchase_days'))
purchase_cat_two_times = purchase_cat.filter(purchase_cat['purchase_days'] > 1)
user_purchase_cat_two_times = purchase_cat_two_times.groupBy(purchase_cat_two_times['cat_id']).\
                            agg(func.countDistinct('user_id').alias('user_purchase_cat_two_times'))

join_cat_purchase = join_brand_purchase.join(user_purchase_cat_two_times,['cat_id'], 'left_outer')
join_cat_purchase = join_cat_purchase.na.fill(0)

#8.4 The number of users who bought on at least two different days from merchants
purchase_merchant = filtered_purchase.groupBy(filtered_purchase['merchant_id'],filtered_purchase['user_id']).\
                            agg(func.countDistinct('day').alias('purchase_days'))
purchase_merchant_two_times = purchase_merchant.filter(purchase_merchant['purchase_days'] > 1)
user_purchase_merchant_two_times = purchase_merchant_two_times.groupBy(purchase_merchant_two_times['merchant_id']).\
                            agg(func.countDistinct('user_id').alias('user_purchase_merchant_two_times'))

join_merchant_purchase = join_cat_purchase.join(user_purchase_merchant_two_times,['merchant_id'], 'left_outer')
join_merchant_purchase = join_merchant_purchase.na.fill(0)

join_merchant_purchase.show()

+-----------+------+--------+-------+------+---------+---+------+-----+-------+----------------------------+-----------------------------+---------------------------+--------------------------------+
|merchant_id|cat_id|brand_id|item_id|action|age_range|day|gender|month|user_id|user_purchase_item_two_times|user_purchase_brand_two_times|user_purchase_cat_two_times|user_purchase_merchant_two_times|
+-----------+------+--------+-------+------+---------+---+------+-----+-------+----------------------------+-----------------------------+---------------------------+--------------------------------+
|        296|  1606|    4870| 398285|     0|        5| 05|     2|   06| 106083|                           0|                            0|                          0|                               0|
|        829|   737|    7719| 321096|     0|        5| 27|     2|   09| 271887|                           0|                            0|                          0|                               0|


In [87]:
#9. Merchant’s market share on the brand:
#the number of purchases of the brand from the merchant/the number of purchases of the brand from all the merchants.

In [88]:
#10. merchant’s user share on the brand:
#the number of users buying the brand from the merchant/the number of buyers of the brand from all the merchants. 

In [89]:
#11. Merchant’s market share on the brand: 
#the number of purchases of the brand from the merchant/the total number of purchases from the merchant

In [ ]:
#12. merchant’s user share on the brand: 
#the number of users buying the brand from the merchant/the total number of buyers of the merchant.